# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 23 2023 9:33AM,258022,10,0086421851,ISDIN Corporation,Released
1,Feb 23 2023 9:33AM,258022,10,0086421849,ISDIN Corporation,Released
2,Feb 23 2023 9:33AM,258022,10,0086421850,ISDIN Corporation,Released
3,Feb 23 2023 9:33AM,258022,10,0086421852,ISDIN Corporation,Released
4,Feb 23 2023 9:33AM,258022,10,0086421853,ISDIN Corporation,Released
5,Feb 23 2023 9:33AM,258022,10,0086421855,ISDIN Corporation,Released
6,Feb 23 2023 9:33AM,258022,10,0086421831,ISDIN Corporation,Released
7,Feb 23 2023 9:33AM,258022,10,0086421832,ISDIN Corporation,Released
8,Feb 23 2023 9:33AM,258022,10,0086421830,ISDIN Corporation,Released
9,Feb 23 2023 9:33AM,258022,10,0086421836,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,258020,Released,1
36,258022,Released,57
37,258023,Released,7
38,258024,Released,1
39,258025,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258020,NaN,NaN,1.0
258022,NaN,NaN,57.0
258023,NaN,NaN,7.0
258024,NaN,NaN,1.0
258025,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257934,0.0,0.0,2.0
257935,2.0,2.0,13.0
257936,0.0,0.0,23.0
257942,0.0,0.0,25.0
257945,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257934,0,0,2
257935,2,2,13
257936,0,0,23
257942,0,0,25
257945,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257934,0,0,2
1,257935,2,2,13
2,257936,0,0,23
3,257942,0,0,25
4,257945,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257934,,,2
1,257935,2,2,13
2,257936,,,23
3,257942,,,25
4,257945,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 23 2023 9:33AM,258022,10,ISDIN Corporation
57,Feb 23 2023 9:32AM,257942,15,"Alliance Pharma, Inc."
82,Feb 23 2023 9:30AM,257936,15,"Alliance Pharma, Inc."
105,Feb 23 2023 9:29AM,258024,16,"SHL Pharma, LLC"
106,Feb 23 2023 9:29AM,258025,10,Emerginnova
111,Feb 23 2023 9:29AM,258023,10,"Digital Brands, LLC"
118,Feb 23 2023 9:26AM,258020,10,"NBTY Global, Inc."
119,Feb 23 2023 9:12AM,258019,10,"Nextsource Biotechnology, LLC"
120,Feb 23 2023 9:11AM,258018,10,Ivaoes Animal Health
121,Feb 23 2023 8:54AM,258017,12,"Uniderm USA, Inc"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 23 2023 9:33AM,258022,10,ISDIN Corporation,,,57
1,Feb 23 2023 9:32AM,257942,15,"Alliance Pharma, Inc.",,,25
2,Feb 23 2023 9:30AM,257936,15,"Alliance Pharma, Inc.",,,23
3,Feb 23 2023 9:29AM,258024,16,"SHL Pharma, LLC",,,1
4,Feb 23 2023 9:29AM,258025,10,Emerginnova,,,5
5,Feb 23 2023 9:29AM,258023,10,"Digital Brands, LLC",,,7
6,Feb 23 2023 9:26AM,258020,10,"NBTY Global, Inc.",,,1
7,Feb 23 2023 9:12AM,258019,10,"Nextsource Biotechnology, LLC",,,1
8,Feb 23 2023 9:11AM,258018,10,Ivaoes Animal Health,,,1
9,Feb 23 2023 8:54AM,258017,12,"Uniderm USA, Inc",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:33AM,258022,10,ISDIN Corporation,57,,
1,Feb 23 2023 9:32AM,257942,15,"Alliance Pharma, Inc.",25,,
2,Feb 23 2023 9:30AM,257936,15,"Alliance Pharma, Inc.",23,,
3,Feb 23 2023 9:29AM,258024,16,"SHL Pharma, LLC",1,,
4,Feb 23 2023 9:29AM,258025,10,Emerginnova,5,,
5,Feb 23 2023 9:29AM,258023,10,"Digital Brands, LLC",7,,
6,Feb 23 2023 9:26AM,258020,10,"NBTY Global, Inc.",1,,
7,Feb 23 2023 9:12AM,258019,10,"Nextsource Biotechnology, LLC",1,,
8,Feb 23 2023 9:11AM,258018,10,Ivaoes Animal Health,1,,
9,Feb 23 2023 8:54AM,258017,12,"Uniderm USA, Inc",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:33AM,258022,10,ISDIN Corporation,57,,
1,Feb 23 2023 9:32AM,257942,15,"Alliance Pharma, Inc.",25,,
2,Feb 23 2023 9:30AM,257936,15,"Alliance Pharma, Inc.",23,,
3,Feb 23 2023 9:29AM,258024,16,"SHL Pharma, LLC",1,,
4,Feb 23 2023 9:29AM,258025,10,Emerginnova,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:33AM,258022,10,ISDIN Corporation,57.0,NaN,NaN
1,Feb 23 2023 9:32AM,257942,15,"Alliance Pharma, Inc.",25.0,NaN,NaN
2,Feb 23 2023 9:30AM,257936,15,"Alliance Pharma, Inc.",23.0,NaN,NaN
3,Feb 23 2023 9:29AM,258024,16,"SHL Pharma, LLC",1.0,NaN,NaN
4,Feb 23 2023 9:29AM,258025,10,Emerginnova,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:33AM,258022,10,ISDIN Corporation,57.0,0.0,0.0
1,Feb 23 2023 9:32AM,257942,15,"Alliance Pharma, Inc.",25.0,0.0,0.0
2,Feb 23 2023 9:30AM,257936,15,"Alliance Pharma, Inc.",23.0,0.0,0.0
3,Feb 23 2023 9:29AM,258024,16,"SHL Pharma, LLC",1.0,0.0,0.0
4,Feb 23 2023 9:29AM,258025,10,Emerginnova,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5675774,257.0,6.0,20.0
12,774044,4.0,17.0,0.0
15,515878,48.0,0.0,0.0
16,516034,2.0,0.0,0.0
18,258009,1.0,0.0,0.0
19,258001,24.0,12.0,0.0
21,773891,2.0,1.0,0.0
22,515995,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5675774,257.0,6.0,20.0
1,12,774044,4.0,17.0,0.0
2,15,515878,48.0,0.0,0.0
3,16,516034,2.0,0.0,0.0
4,18,258009,1.0,0.0,0.0
5,19,258001,24.0,12.0,0.0
6,21,773891,2.0,1.0,0.0
7,22,515995,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,257.0,6.0,20.0
1,12,4.0,17.0,0.0
2,15,48.0,0.0,0.0
3,16,2.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,24.0,12.0,0.0
6,21,2.0,1.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,257.0
1,12,Released,4.0
2,15,Released,48.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21,22
Status,,,,,,,,
Completed,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,6.0,17.0,0.0,0.0,0.0,12.0,1.0,0.0
Released,257.0,4.0,48.0,2.0,1.0,24.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21,22
0,Completed,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,17.0,0.0,0.0,0.0,12.0,1.0,0.0
2,Released,257.0,4.0,48.0,2.0,1.0,24.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21,22
0,Completed,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,17.0,0.0,0.0,0.0,12.0,1.0,0.0
2,Released,257.0,4.0,48.0,2.0,1.0,24.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()